# Structure Optimization

In [ ]:
%matplotlib notebook
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib import animation

import random
import time

In [ ]:
from potentials import gradients
from distances import vectors
from ewald_numpy import Ewald

In [ ]:
def grad_Ewald( x, q, boxsz=(0,1), pbc=False ):
    vecs = vectors(x, boxsz)
    E = Ewald(charges=q, boxsize=boxsz, alpha=5, r_cutoff=5, n_max=2, k_cutoff=5)
    return -E.force(x)

def grad_HLC( x, q, boxsize=(0,1), pbc=False ):
    """Sum of Harmonic, LJ and Coulombs gradients"""
    vecs = vectors(x, boxsize, pbc)
    return gradients.harmonic(x, boxsize) + gradients.LJ(vecs) - gradients.coulomb(vecs, q)
    
def descent( x, q, grad, a=1e-4, prec=1e-10, maxst=1e6, k=.1, boxsize=(0, 1), pbc=False ):
    """Gradient Descent
    
    Arguments:
        x    (float): position vectors (dim = n x 3)
        q: charge
        a    (float): 'learning rate' alpha = 1e-4
        prec (float): difference between steps, precision = 1e-10
        maxst  (int): max # of steps, maxst = 1e6
        k: factor harmonic pot
    
    Output:
        x: position array,
        step: # of steps needed to converge"""
    xmin, xmax = boxsize[0], boxsize[1]
    L = xmax - xmin
    x = x[None, :, :]
    step = 0
    f = grad(x[-1], q, pbc=pbc)
    x1 = x[-1] - a * f
    if pbc:
        x1 = x1 + (x1 < xmin) * L
        x1 = x1 - (x1 > xmax) * L  
    while step < maxst and np.linalg.norm(x[-1] - x1) > prec:
        x = np.append(x, x1[None, :, :], axis=0)
        f = grad(x[-1], q, pbc=pbc)
        x1 = x[-1] - a * f
        if pbc:
            x1 = x1 + (x1 < xmin) * L
            x1 = x1 - (x1 > xmax) * L
        step += 1
    return x, step

# 2D case, Harmonic + LJ + Coulomb

In [ ]:
def x_init_gen2D(N=3, qfac=7, alternating=False): 
    """Generate initial particle distribution
        N**2 - # of particles
        qfac - 
    
    Output:
        x: position array,
        step: # of steps needed to converge"""
    x, y = np.linspace(-N, N, N), np.linspace(-N, N, N)
    XX, YY = np.meshgrid(x,y)
    if alternating:
        q = np.array([(-1)**i for i in range(N) for j in range(N)])
    else:
        q = np.array([[-1,1][random.randrange(2)] for i in range(N*N)])
    q *= qfac
    x_init = np.array([XX.flatten(),YY.flatten()]).T
    x_init += np.random.uniform(low=-0.5, high=0.5, size=(N**2,2))
    #x_init *= 10
    assert len(x_init) == N**2
    return x_init, q

In [ ]:
x_init, q = x_init_gen2D()
x_init = np.array([[-1.02180131,  1.82156929], [-1.80851221, -0.79108905],
                   [ 1.45298718,  0.53496423], [-1.33659605, -1.4763313 ]])
q = np.array([ 1, -1,  1, -1])
print(x_init.shape, q)

In [ ]:
print(x_init)

In [ ]:
print('(# of particles, dimensions): ', x_init.shape)
t0 = time.time()
positions, nsteps = descent(x_init, q, grad_HLC, a=1e-4, prec=1e-5, maxst=40000, k=0.1, pbc=False)
t1 = time.time()
print('# of steps:', nsteps, '| positions.shape:', positions.shape)
print('time elapsed: ', t1 - t0)

In [ ]:
mask = np.count_nonzero(np.linalg.norm(positions[-1], axis=-1) < 10)
print(mask)

fig, ax = plt.subplots(figsize=(9, 9))
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], c=q)
circles = [plt.Circle(r, radius=0.5, fill=False) 
            for i,r in enumerate(x_init)]
for c in circles:
    plt.gca().add_patch(c)
ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
def animate(i):
    index = 4*i
    data = positions[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    return scat
anim = animation.FuncAnimation(fig, animate, interval=1)

In [ ]:
def x_init_gen3D(N=3): 
    x = np.linspace(-N, N, N)
    XX, YY, ZZ = np.meshgrid(x,x,x)
    #q = np.array([[-1,1][random.randrange(2)] for i in range(N**3)])
    q = np.array([(-1)**j for i in range(N**2) for j in range(N)]) * 10
    x_init = np.array([XX.flatten(),YY.flatten(),ZZ.flatten()]).T
    x_init += np.random.uniform(low=-0.1, high=0.1, size=(N**3,3))
    #x_init *= 10
    assert len(x_init) == N**3
    return x_init, q

In [ ]:
x_init, q = x_init_gen3D(2)
print(x_init.shape)
positions, nsteps = descent(x_init, q, grad_HLC, k=0.1, 
                       a=1e-4, prec=1e-6, maxst=80000)
print('# of steps:', nsteps, '| positions.shape:', positions.shape)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(9.5,9.5))
ax = fig.add_subplot(111, projection='3d')
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], x_init[:,2], c=q)
#circles = [plt.Circle(r, radius=0.5, fill=False) 
#            for i,r in enumerate(x_init)]
#for c in circles:
#    plt.gca().add_patch(c)
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
def animate(i):
    index = 4*i
    data = positions[index]
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    #for i, c in enumerate(circles):
    #    c.center = data[i]
    return scat,

#anim = animation.FuncAnimation(fig, animate, interval=1)
anim = animation.FuncAnimation(fig, animate, interval=20, frames=1050, repeat=False)

# 3D case, Ewald forces

In [ ]:
x_init, q = x_init_gen3D(2)
print(x_init.shape)
print(q)

In [ ]:
t0 = time.time()
positions, nsteps = descent(x_init, q, grad_Ewald, a=1e-4, prec=1e-6, maxst=10000)
t1 = time.time()
print('# of steps:', nsteps, '| positions.shape:', positions.shape)
print('time: ', t1 - t0)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(9.5,9.5))
ax = fig.add_subplot(111, projection='3d')
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], x_init[:,2], c=q)
#circles = [plt.Circle(r, radius=0.5, fill=False) 
#            for i,r in enumerate(x_init)]
#for c in circles:
#    plt.gca().add_patch(c)
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
def animate(i):
    index = 4*i
    data = positions[index]
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    #for i, c in enumerate(circles):
    #    c.center = data[i]
    return scat,


#anim = animation.FuncAnimation(fig, animate, interval=1)
anim = animation.FuncAnimation(fig, animate, interval=20, frames=1050, repeat=False)

In [ ]:
boxsize = (0, 1)
xmin, xmax = boxsize[0], boxsize[1]
L = xmax - xmin
pbc = True
x0 = np.array([[.4, .6], [.1, .8]])
f = np.array([[.5, -.1], [.1, -.7]])
x1 = x0 - f
print(x0)
print(f)
print(x1)
print(x1 < xmin)
print(x1 > xmax)
if pbc:
    x1 = x1 + (x1 < xmin) * L
    x1 = x1 - (x1 > xmax) * L
print(x1)

In [ ]:
import numpy as np

In [ ]:
def pbc( x1, box ):
    print(x1)
    xmin, xmax = box[0], box[1]
    assert xmin < xmax
    L = xmax - xmin
    x1[x1 < xmin] = xmax - (xmin - x1[x1 < xmin]) %  L
    x1[x1 > xmax] = xmin + (x1[x1 > xmax] - xmax) %  L
    print(x1)
    return

# box = (+, +)
#pp = np.array([-1.2, 2.4, 4, 5.3, 8.7])
#pbc(pp, box=(3, 5))
# box = (-, -)
#mm = np.array([1.1, -4.8, -6., -7.3, -10.7])
#pbc(mm, box=(-7, -5))
# box = (-, +)
#mp = np.array([-10.4, -2.2, -.1, 3.3, 9.7])
#pbc(mp, box=(-2, 3))

In [ ]:
kc = 3
kvec = np.array([[i,j, k] 
                 for i in range(kc + 1) 
                 for j in range(int(np.sqrt(kc**2 - i**2)) + 1) 
                 for k in range(int(np.sqrt(kc**2 - i**2 - j**2)) + 1) if (i,j,k) != (0,0,0)])
print(kvec)